In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
import gc
import time

pd.set_option('display.max_columns', 999)

In [2]:
raw_df = pd.read_csv('featured_data.csv', index_col='datetime', parse_dates=True)
raw_df.describe()

,machineID,error2_rollingmean24,error5_rollingmean24,error1_rollingmean24,error3_rollingmean24,error4_rollingmean24,sincelastcomp3,sincelastcomp2,sincelastcomp4,sincelastcomp1,age,volt_rollingmean_12,volt_rollingstd_12,rotate_rollingmean_12,rotate_rollingstd_12,pressure_rollingmean_12,pressure_rollingstd_12,vibration_rollingmean_12,vibration_rollingstd_12,volt_rollingmean_24,volt_rollingstd_24,rotate_rollingmean_24,rotate_rollingstd_24,pressure_rollingmean_24,pressure_rollingstd_24,vibration_rollingmean_24,vibration_rollingstd_24,volt_rollingmean_36,volt_rollingstd_36,rotate_rollingmean_36,rotate_rollingstd_36,pressure_rollingmean_36,pressure_rollingstd_36,vibration_rollingmean_36,vibration_rollingstd_36,failure_comp1,failure_comp2,failure_comp3,failure_comp4
count,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000,730000.000000
mean,500.500000,0.000367,0.000154,0.000312,0.000350,0.000182,31.346332,31.228510,32.124236,31.517948,9.862000,170.525211,14.061870,445.997791,46.972041,100.698365,9.421627,40.293811,4.700074,170.290104,14.564086,445.393748,48.705687,100.559606,9.798922,40.238007,4.878768,170.055410,14.740632,444.786852,49.357812,100.421026,9.958281,40.182378,4.948698,0.035148,0.048071,0.018904,0.022970
std,288.675188,0.003902,0.002525,0.003601,0.003812,0.002749,35.152047,34.981629,36.715142,35.499147,6.089746,8.513457,3.099596,26.876932,10.393774,6.422975,2.151708,2.733828,1.049798,10.083188,2.337919,29.481904,7.939878,7.019246,1.746543,2.893024,0.819126,11.703056,2.079465,32.920558,7.188817,7.745928,1.681495,3.154339,0.758761,0.184154,0.213917,0.136187,0.149807
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,250.750000,0.000000,0.000000,0.000000,0.000000,0.000000,5.000000,5.000000,5.000000,5.000000,5.000000,167.214493,11.930060,438.380461,39.838755,98.131262,7.964623,39.069133,3.982985,168.041810,13.066296,441.504082,43.626509,98.683274,8.718817,39.343257,4.361342,168.402731,13.514040,442.832542,45.122115,98.930280,9.016594,39.466704,4.508394,0.000000,0.000000,0.000000,0.000000
50%,500.500000,0.000000,0.000000,0.000000,0.000000,0.000000,20.000000,20.000000,21.000000,20.000000,10.000000,170.236916,13.949606,449.029399,46.593409,100.140831,9.317476,40.074698,4.656659,170.187657,14.523077,449.164918,48.491410,100.115628,9.699838,40.061499,4.847905,170.177547,14.726414,449.190706,49.172576,100.106046,9.831026,40.054979,4.914001,0.000000,0.000000,0.000000,0.000000
75%,750.250000,0.000000,0.000000,0.000000,0.000000,0.000000,45.000000,45.000000,46.000000,46.000000,15.000000,173.382387,16.078487,459.139629,53.691721,102.230183,10.753834,41.121748,5.369515,172.443308,16.043273,456.363967,53.608140,101.612025,10.734068,40.811144,5.362771,172.043313,15.984011,455.124559,53.471010,101.345533,10.700100,40.679778,5.344830,0.000000,0.000000,0.000000,0.000000
max,1000.000000,0.083333,0.041667,0.083333,0.083333,0.083333,265.000000,311.000000,299.000000,322.000000,20.000000,225.106859,34.149485,516.419586,113.152459,166.739169,28.789963,63.894733,12.570563,224.651654,28.510334,498.664658,111.542785,162.459319,28.040488,62.664375,11.772613,223.445932,30.325583,488.459530,109.705328,157.606447,30.073329,62.671135,12.872517,1.000000,1.000000,1.000000,1.000000


In [3]:
excluded_list = ['model_encoded', 'machineID']
label_list = ['failure_comp1', 'failure_comp2', 'failure_comp3', 'failure_comp4']
s = set(excluded_list + label_list)
print('exclude columns:', set(excluded_list + label_list))

exclude columns: {'failure_comp2', 'failure_comp4', 'model_encoded', 'failure_comp1', 'machineID', 'failure_comp3'}


In [4]:
feature_list = [x for x in raw_df.columns if x not in set(excluded_list + label_list)]
print('feature columns:', feature_list)

feature columns: ['error2_rollingmean24', 'error5_rollingmean24', 'error1_rollingmean24', 'error3_rollingmean24', 'error4_rollingmean24', 'sincelastcomp3', 'sincelastcomp2', 'sincelastcomp4', 'sincelastcomp1', 'age', 'volt_rollingmean_12', 'volt_rollingstd_12', 'rotate_rollingmean_12', 'rotate_rollingstd_12', 'pressure_rollingmean_12', 'pressure_rollingstd_12', 'vibration_rollingmean_12', 'vibration_rollingstd_12', 'volt_rollingmean_24', 'volt_rollingstd_24', 'rotate_rollingmean_24', 'rotate_rollingstd_24', 'pressure_rollingmean_24', 'pressure_rollingstd_24', 'vibration_rollingmean_24', 'vibration_rollingstd_24', 'volt_rollingmean_36', 'volt_rollingstd_36', 'rotate_rollingmean_36', 'rotate_rollingstd_36', 'pressure_rollingmean_36', 'pressure_rollingstd_36', 'vibration_rollingmean_36', 'vibration_rollingstd_36']


In [5]:
print('sainty check')
_,n = raw_df.shape
print("{} columns in raw data,\n{} columns for feature".format(n, len(feature_list)))

sainty check
40 columns in raw data,
34 columns for feature


In [6]:
split_time_point = '2015-10-30'
df_train = raw_df[raw_df.index <= split_time_point]
df_test = raw_df[raw_df.index > split_time_point]
print("raw data shape {},  train data shape {}, test data shape{}".format(raw_df.shape, df_train.shape, df_test.shape))

raw data shape (730000, 40),  train data shape (604000, 40), test data shape(126000, 40)


In [7]:
x_train = df_train[feature_list].get_values()
y_train = df_train[label_list].get_values()
x_test = df_test[feature_list].get_values()
y_test = df_test[label_list].get_values()

In [8]:
print('x_train shape:', x_train.shape)
print('y_train shape:', y_train.shape)
print('x_test  shape:', x_test.shape)
print('y_test  shape:', y_test.shape)

x_train shape: (604000, 34)
y_train shape: (604000, 4)
x_test  shape: (126000, 34)
y_test  shape: (126000, 4)


In [9]:
y_test[0]

array([0., 0., 0., 1.])

In [10]:
dt_clf = DecisionTreeClassifier(max_depth=15)

start_time = time.time()

dt_clf.fit(x_train, y_train)

elapsed_time = time.time() - start_time
print("\nComplete", time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))


Complete 00:00:43


In [11]:
y_pred = dt_clf.predict(x_test)
print(dt_clf.score(x_test, y_test))

0.9092142857142858


In [12]:
print("Confusion Matrix")
for i in range(4):
    print("component {}".format(i))
    print(classification_report(y_test[:,i], y_pred[:,i]))

Confusion Matrix
component 0
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99    121908
         1.0       0.85      0.24      0.37      4092

   micro avg       0.97      0.97      0.97    126000
   macro avg       0.91      0.62      0.68    126000
weighted avg       0.97      0.97      0.97    126000

component 1
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98    120306
         1.0       0.80      0.25      0.38      5694

   micro avg       0.96      0.96      0.96    126000
   macro avg       0.88      0.62      0.68    126000
weighted avg       0.96      0.96      0.95    126000

component 2
              precision    recall  f1-score   support

         0.0       0.99      1.00      0.99    123662
         1.0       0.73      0.26      0.38      2338

   micro avg       0.98      0.98      0.98    126000
   macro avg       0.86      0.63      0.69    126000
weighted avg       0.

In [13]:
rf_clf = RandomForestClassifier(n_estimators=100, max_depth=15)

start_time = time.time()

rf_clf.fit(x_train, y_train)

elapsed_time = time.time() - start_time
print("\nComplete", time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))


Complete 00:09:18


In [14]:
y_pred = rf_clf.predict(x_test)
print(rf_clf.score(x_test, y_test))

0.9123015873015873


In [15]:
print("Confusion Matrix")
for i in range(4):
    print("component {}".format(i))
    print(classification_report(y_test[:,i], y_pred[:,i]))

Confusion Matrix
component 0
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.99    121908
         1.0       0.96      0.22      0.36      4092

   micro avg       0.97      0.97      0.97    126000
   macro avg       0.97      0.61      0.67    126000
weighted avg       0.97      0.97      0.97    126000

component 1
              precision    recall  f1-score   support

         0.0       0.97      1.00      0.98    120306
         1.0       0.94      0.25      0.39      5694

   micro avg       0.97      0.97      0.97    126000
   macro avg       0.95      0.62      0.69    126000
weighted avg       0.96      0.97      0.96    126000

component 2
              precision    recall  f1-score   support

         0.0       0.98      1.00      0.99    123662
         1.0       0.98      0.19      0.31      2338

   micro avg       0.98      0.98      0.98    126000
   macro avg       0.98      0.59      0.65    126000
weighted avg       0.